# Librerias

In [1]:
import sys
import cv2
import dlib
import os
import numpy as np
import keras
import time
from statistics import mean
from collections import Counter
import pandas as pd
import pyshine as ps

Warning! No module named 'soundfile'


# Parametros

### Resolución de imagen que se muestra

In [2]:
show_height = None
show_width = 950

### Resolución de imagen a procesar

In [3]:
height = 500
width = None

### Resolución de rasgos faciales
tamaño de procesamiento

#### Ojo

In [4]:
ancho_ojo = 90
alto_ojo = 90
eye_input_shape = (alto_ojo, ancho_ojo)

#### Boca

In [5]:
ancho_boca = 64
alto_boca = 64
mouth_input_shape = (ancho_boca, alto_boca)

### ¿Cuantos frames se van a saltar?

In [6]:
n_skipping_frames = 1
framesSkipping = n_skipping_frames + 1

### Umbrales

In [7]:
# Umbral de frames para realizar mediciones
original_threshold = 200

# Umbral para determinar si una secuencia de boca abierta se trata de un bostezo
uba = 50

# Umbral de ojo cerrado
closeeye_threshold = 7

# Maximo valor PERCLOS permitido
mvpp = 8

# Cada cuantos frames se evalua el desempeño parcial del video
unidad_accuracy = 620

# Modelos pre entrenados

### Reconocimiento de rostro & puntos faciales

In [8]:
# Detector caras HOG+SVM
detector = dlib.get_frontal_face_detector()
# Detector de puntos de interes
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

### Estado de ojo

In [9]:
eye_model = keras.models.load_model("final_model_eye.h5")

### Estado de boca

In [10]:
mouth_model = keras.models.load_model("final_model_mouth.h5")

# Funciones Auxiliares

### Modificar tamaño de imagen

In [11]:
def image_resize(image, width = None, height = None):
    # Obtener ancho y alto
    dim = None
    (h, w) = image.shape[:2]

    # Tamaño de imagen original
    if width is None and height is None:
        return w, h

    if width is None:
        # calcular el ancho usando el alto
        r = height / float(h)
        width = int(w * r)
        # fator
        frame_resize_scale = float(h)/height
    else:
        # calcular el alto usando el ancho
        r = width / float(w)
        height = int(h * r)
        # fator
        frame_resize_scale = float(w)/width

    return width, height, frame_resize_scale

### Segmentado de cara

In [12]:
def facedetector(frame, landmarks):
    x1 = landmarks.part(0).x
    y1 = landmarks.part(19).y
    x2 = landmarks.part(15).x
    y2 = landmarks.part(8).y
    
    return x1, x2, y1, y2

In [13]:
def facedetector2(face):
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    
    return x1, x2, y1, y2

### Segmentado de ojo derecho

In [14]:
def eyedetector(frame, landmarks, height):
    x1 = landmarks.part(18).x - 3
    x2 = landmarks.part(21).x + 1
    y1 = landmarks.part(21).y
    y2 = landmarks.part(30).y
    
    return x1, x2, y1, y2

def eyedetector(frame, landmarks, height):
    x1 = landmarks.part(18).x - 3
    x2 = landmarks.part(21).x + 1
    y1 = landmarks.part(38).y - int((height * 0.11))
    y2 = landmarks.part(40).y + int((height * 0.14))
    
    return x1, x2, y1, y2

### Segmentado de boca

In [15]:
def mouthdetector(frame, landmarks, height):
    x1 = landmarks.part(6).x
    x2 = landmarks.part(10).x 
    y1 = landmarks.part(33).y 
    y2 = landmarks.part(6).y - int((height * 0.05))
    
    return x1, x2, y1, y2

### Escalar coordenadas de imagen pequeña a imagen a mostrar

In [16]:
def show_coods(x1, x2, y1, y2, escala):
    x1 = int(x1 * escala)
    x2 = int(x2 * escala)
    y1 = int(y1 * escala)
    y2 = int(y2 * escala)

    return x1, x2, y1, y2

# Detector de Cansancio

In [17]:
def detector_cansancio(show_height, show_width, height, width, eye_input_shape, mouth_input_shape, framesSkipping,
                      original_threshold, uba, closeeye_threshold, mvpp, detector, predictor, eye_model,
                       mouth_model, fuente, clase_real):
    
    
    # Tipo de letra
    imprimir = 0

    # Inicializar lista de caras vacia
    faces = []

    # Estatus de ojo no detectados
    eye_status = "Undetected eye"
    mouth_status = "Undetected mouth"
    
        
    cap = cv2.VideoCapture(fuente)


    ret, frame = cap.read()

    # Obtener dimensiones y escala para frames
    if ret == True:
        show_width, show_height, _ = image_resize(frame, height = show_height, width = show_width)
        frame = cv2.resize(frame, (show_width, show_height), interpolation = cv2.INTER_AREA)
        width, height, escala = image_resize(frame, height = height, width = width)

    else:
        print("No se puede leer el frame")
        sys.exit()


    # FPS 
    framePerSecond = 20.0
    tyme = cv2.getTickCount()
    count = 0
    pfps2 = []


    # Inicializar contadores
    # --------------------------
    # Contador de frames para ojo cerrado
    perclos_frames = 0

    # Contador de frames para periodo de medicion PERCLOS
    n_frames = 0
    
    # Contador de frames totales
    frames_totales = 0
    frames_totales2 = 0
    
    # Contador para medir frames continuos de ojo cerrado
    ojo_continuo = 0

    # Tiempo continuo de ojo cerrado corregido por frames saltados
    TCOC = 0

    # Contador para medir frames continuos de boca abierta
    fba = 0

    # Para guardar la cantidad de bostezos en una determinada ventana de tiempo
    bandera_perclos = 0
    bandera_ojo_cerrado = 0
    bandera_bostezo = 0

    # Inicializar umbral
    #-----------------------
    threshold = original_threshold
    
    # Lista para guardar estado de cansancio a traves del tiempo
    vector_cansancio = []
    vector_cansancio2 = []
    
    # Clasificador
    data = [[] for _ in range(5)]
    video_name = []
    perclos_prom = []
    ojoCerrado_prom = []
    n_bostezos = 0



    tiempoc = time.time()


    while True:


        # Empezar cuenta para calcular FPS
        if count == 0:
            tyme = cv2.getTickCount()

        # obtener frame del video
        ret, frame = cap.read()

        # no hay error al obtener frame
        if ret == True:
            
            #---------------------------------------
            # Para determinar accuracy en el video
            #---------------------------------------
            if frames_totales == unidad_accuracy:
                
                
                data[0].append('o')
                data[1].append(clase_real)
                perclos_mean = mean(perclos_prom)
                data[2].append(perclos_mean)
                ojo_mean = mean(ojoCerrado_prom)
                data[3].append(ojo_mean)
                data[4].append(n_bostezos)
                perclos_prom = []
                ojoCerrado_prom = []
                n_bostezos = 0
         
                if bandera_perclos >= 1:
                    vector_cansancio2.append('perclos')
                if bandera_ojo_cerrado >= 1:
                    vector_cansancio2.append('ojo_c')
                if bandera_bostezo >= 1:
                    vector_cansancio2.append('bostezo')
                    
                if (bandera_perclos >= 1 or bandera_bostezo >= 1 or bandera_ojo_cerrado >= 1):
                    vector_cansancio.append('cansado')
                else:
                    vector_cansancio.append('alerta')
                    vector_cansancio2.append('alerta')
                
                frames_totales = 0
                bandera_perclos = 0
                bandera_ojo_cerrado = 0
                bandera_bostezo = 0


            # Cambiar resolucion de frame
            frame = cv2.resize(frame, (show_width, show_height), interpolation = cv2.INTER_AREA)


            # Crear frame pequeño
            frame_small = cv2.resize(frame, (width, height), interpolation = cv2.INTER_AREA)


            # Cambiar frame pequeño a escala de grises
            gray = cv2.cvtColor(frame_small, cv2.COLOR_BGR2GRAY)

            # Detectar cara en frame pequeño si es un frame que no debe saltarse
            if (count % framesSkipping == 0):
                faces = detector(gray)

                # Verificar que se detecto una cara
                if len(faces) >= 1:
                    for face in faces:
                        landmarks = predictor(gray, face)

                        # Segmentado de cara
                        #x1, x2, y1, y2 = facedetector2(face)
                        x1, x2, y1, y2 = facedetector(gray, landmarks)
                        face_height = y2 - y1

                        # Enmarcado en frame para mostrar
                        x1, x2, y1, y2 = show_coods(x1, x2, y1, y2, escala)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)

                        # Segmentado amplio de boca
                        x1, x2, y1, y2 = mouthdetector(gray, landmarks, face_height)
                        # Imagen de boca para procesar
                        boca_g = gray[y1:y2, x1:x2]
                        # Enmarcado en frame para mostrar
                        x1, x2, y1, y2 = show_coods(x1, x2, y1, y2, escala)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

                        # Segmentado amplio de ojo derecho
                        x1, x2, y1, y2 = eyedetector(gray, landmarks, face_height)
                        # Imagen de ojo para procesar
                        ojo_g = gray[y1:y2, x1:x2]
                        # Enmarcado en frame para mostrar
                        x1, x2, y1, y2 = show_coods(x1, x2, y1, y2, escala)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)


                        # ---------------------------------------
                        #    Preprocesamiento
                        # ---------------------------------------

                        # ojo
                        # ajustar tamaño de imagen
                        try:
                            ojo_g = cv2.resize(ojo_g, eye_input_shape, interpolation = cv2.INTER_AREA)

                            # ajustar dimensiones (blanco y negro)
                            ojo_der = np.expand_dims(ojo_g, axis=0)
                            ojo_der = np.expand_dims(ojo_der, axis=3)

                            # Normalización
                            ojo_der = ojo_der / 255.0
                        except:
                            continue

                        # boca
                        # ajustar tamaño de imagen
                        try:
                            boca_g = cv2.resize(boca_g, mouth_input_shape, interpolation = cv2.INTER_AREA)
                            
                            # ajustar dimensiones (blanco y negro)
                            boca = np.expand_dims(boca_g, axis=0)
                            boca = np.expand_dims(boca, axis=3)

                            # Normalización
                            boca = boca / 255.0
                        except:
                            continue

                        # ---------------------------------------
                        #  Deteccion de estado del ojo
                        # ---------------------------------------

                        # Determinar si el ojo esta abierto o cerrado
                        eye_prediction = eye_model.predict(ojo_der)

                        if eye_prediction >= 0.5:
                            eye_status = "Close Eye"
                            #ps.putBText(frame,  'PARPADEO', text_offset_x = 10, text_offset_y = 120,
                            #vspace = 5, hspace = 10, font_scale = 6, background_RGB = (255, 255, 255),
                            #text_RGB = (0, 0, 0), thickness = 1, alpha = 0.5)

                            #---------------------------------------
                            #  Detección de cansancio PERCLOS
                            #---------------------------------------
                            # Actualizar contador de frames ojo cerrado
                            perclos_frames = perclos_frames + 1

                            # Verificar si se ha alcanzado la unidad de tiempo PERCLOS
                            if n_frames >= threshold:
                                # Calcular PERCLOS
                                perclos = ((perclos_frames * framesSkipping) / n_frames) * 100
                                perclos_prom.append(perclos)
                                
                                
                                if imprimir == 1:
                                    print(perclos)
                                # Se detecta cansacio a traves de MVPP
                                if perclos >= mvpp:
                                    if imprimir == 1:
                                        print("Se detecta cansancio, se ha superado el limite PERCLOS")
                                        print("PERCLOS: ", perclos)
                                    bandera_perclos = bandera_perclos + 1

                                # Reiniciar contadores de frames transcurridos y frames ojo cerrado
                                n_frames = 0
                                perclos_frames = 0


                            #----------------------------------------
                            #  Decición tiempo continuo ojo cerrado
                            #----------------------------------------

                            # Aumentar el contador de ojo continuamente cerrado
                            ojo_continuo = ojo_continuo + 1
                            # correccion por frames saltados
                            TCOC = ojo_continuo * framesSkipping


                        else:
                            
                            ojoCerrado_prom.append(TCOC)
                            eye_status = "Open Eye"
                            # Reiniciar contador de ojo continuamente cerrado
                            ojo_continuo = 0

                            #---------------------------------------
                            #  Detección de cansancio PERCLOS
                            #---------------------------------------

                            # Verificar si se ha alcanzado la unidad de tiempo PERCLOS
                            if n_frames >= threshold:
                                # Calcular PERCLOS
                                perclos = ((perclos_frames * framesSkipping) / n_frames) * 100
                                perclos_prom.append(perclos)
                                if imprimir == 1:
                                    print(perclos)
                                # Se detecta cansacio a traves de MVPP
                                if perclos >= mvpp:
                                    if imprimir == 1:
                                        print("Se detecta cansancio, se ha superado el limite PERCLOS")
                                        print("PERCLOS: ", perclos)
                                    bandera_perclos = bandera_perclos + 1

                                # Reiniciar contadores de frames transcurridos y frames ojo cerrado
                                n_frames = 0
                                perclos_frames = 0


                        # ---------------------------------------
                        #  Deteccion de estado de la boca
                        # ---------------------------------------

                        # Determinar si la boca esta abierta o cerrada
                        mouth_prediction = mouth_model.predict(boca)

                        # Si la boca esta cerrada
                        if mouth_prediction >= 0.5:
                            mouth_status = "Close Mouth"
                            # Reiniciar contador de boca abierta
                            fba = 0


                        # Si la boca esta abierta
                        else:
                            mouth_status = "Open Mouth"
                            fba = fba + 1
                            # Corregir por frames que se saltan
                            yawn = fba * framesSkipping

                            # Determinar si se detecta un bostezo
                            if yawn >= uba:
                                n_bostezos = n_bostezos + 1
                                if imprimir == 1:
                                    print("Se detecta cansancio, se detecto un bostezo")
                                bandera_bostezo = bandera_bostezo + 1

                                # reiniciar contador de frames boca abierta
                                fba = 0



                # Si no se detecto ninguna cara
                else:
                    eye_status = "Undetected eye"
                    perclos_frames = perclos_frames + 1
                    mouth_status = "Undetected mouth"



                # FPS
                # Contador global de frames
                n_frames = n_frames + 1
                frames_totales = frames_totales + 1
                frames_totales2 = frames_totales2 + 1

                # Calcular fps
                count = count + 1
                if (count == framesSkipping):
                    pfps2.append(framePerSecond)
                    tyme = (cv2.getTickCount() - tyme)/cv2.getTickFrequency()
                    framePerSecond = (framesSkipping)/tyme
                    count = 0



                # FPS
                ps.putBText(frame,  "{0:.2f}-FPS".format(framePerSecond), text_offset_x = 10, text_offset_y = 20,
                           vspace = 5, hspace = 10, font_scale = 0.5, background_RGB = (255, 255, 255),
                           text_RGB = (0, 0, 0), thickness = 1, alpha = 0.4)

                #---------------------------


                # Estado del ojo
                ps.putBText(frame,  eye_status, text_offset_x = 10, text_offset_y = 43,
                           vspace = 6, hspace = 10, font_scale = 0.65, background_RGB = (255, 255, 255),
                           text_RGB = (0, 0, 0), thickness = 1, alpha = 0.4)
                

                # Estado de la boca
                ps.putBText(frame,  mouth_status, text_offset_x = 10, text_offset_y = 70,
                           vspace = 6, hspace = 10, font_scale = 0.65, background_RGB = (255, 255, 255),
                           text_RGB = (0, 0, 0), thickness = 1, alpha = 0.4)
                

                cv2.imshow('frame', frame)

            # Si el frame se debe saltar
            else:

                # Verificar que se detecto una cara
                if len(faces) >= 1:
                    for face in faces:

                        landmarks = predictor(gray, face)

                        # Segmentado de cara
                        #x1, x2, y1, y2 = facedetector2(face)
                        x1, x2, y1, y2 = facedetector(gray, landmarks)
                        face_height = y2 - y1

                        # Enmarcado en frame para mostrar
                        x1, x2, y1, y2 = show_coods(x1, x2, y1, y2, escala)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)

                        # Segmentado amplio de boca
                        x1, x2, y1, y2 = mouthdetector(gray, landmarks, face_height)
                        # Enmarcado en frame para mostrar
                        x1, x2, y1, y2 = show_coods(x1, x2, y1, y2, escala)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

                        # Segmentado amplio de ojo derecho
                        x1, x2, y1, y2 = eyedetector(gray, landmarks, face_height)
                        # Enmarcado en frame para mostrar
                        x1, x2, y1, y2 = show_coods(x1, x2, y1, y2, escala)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

                # Si no se detecto ninguna cara
                else:
                    eye_status = "Undetected eye"
                    #perclos_frames = perclos_frames + 1
                    mouth_status = "Undetected mouth"



                # FPS
                # Contador global de frames
                n_frames = n_frames + 1
                frames_totales = frames_totales + 1
                frames_totales2 = frames_totales2 + 1

                # Calcular fps
                count = count + 1
                if (count == framesSkipping):
                    pfps2.append(framePerSecond)
                    tyme = (cv2.getTickCount() - tyme) / cv2.getTickFrequency()
                    framePerSecond = (framesSkipping) / tyme
                    count = 0



                # FPS metodo 2
                ps.putBText(frame,  "{0:.2f}-FPS".format(framePerSecond), text_offset_x = 10, text_offset_y = 20,
                           vspace = 5, hspace = 10, font_scale = 0.5, background_RGB = (255, 255, 255),
                           text_RGB = (0, 0, 0), thickness = 1, alpha = 0.4)

                # ---------------------------


                # Conservar estado del ojo de frame anterior
                ps.putBText(frame,  eye_status, text_offset_x = 10, text_offset_y = 43,
                           vspace = 6, hspace = 10, font_scale = 0.65, background_RGB = (255, 255, 255),
                           text_RGB = (0, 0, 0), thickness = 1, alpha = 0.4)
                    

                # Conservar estado de la boca del frame anterior
                ps.putBText(frame,  mouth_status, text_offset_x = 10, text_offset_y = 70,
                           vspace = 6, hspace = 10, font_scale = 0.65, background_RGB = (255, 255, 255),
                           text_RGB = (0, 0, 0), thickness = 1, alpha = 0.4)
                

                cv2.imshow('frame', frame)

        else:
            print("El video terminó")
            break



        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
        if frames_totales2 >= 6700:
            break


    finalt = time.time()
    ttf = finalt-tiempoc
    realfps = n_frames/ttf
    #print("tiempo corriendo:", ttf)
    fps_prom = mean(pfps2)
    #print("FPS promedio: ", mean(pfps2))
    cap.release()
    cv2.destroyAllWindows()
    
    return vector_cansancio, vector_cansancio2, fps_prom, data

# Probar el modelo

In [18]:
directory = r'D:\Datasets\UTA\dataset2'
classes = ['cansado', 'alerta']


In [19]:
resultado = {}
cansado = 0
clase_predicha = None
aciertos = 0
num_patrones = 0
equivocaciones = 0
data = [[] for _ in range(6)]
data2 = [[] for _ in range(5)]
clase_real = 'alerta'

vector_cansancio, vector_cansancio2, fps, datos = detector_cansancio(show_height, show_width, height, width, eye_input_shape, mouth_input_shape, framesSkipping,
                          original_threshold, uba, closeeye_threshold, mvpp, detector, predictor, eye_model,
                           mouth_model, 0, clase_real)
